In [93]:
import pandas as pd
import numpy as np
import json
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
import requests
from requests_oauthlib import OAuth1
from os.path import expanduser
import time
home = expanduser("~")

In [10]:
tweets_data_path = '/Users/nathanbackblaze/DS/Metis/projects/Final Projects/AWS_data/fetched_tweets_june20_noon_pst.txt'
results = []
tweets_file = open(tweets_data_path, "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        results.append(tweet)
    except:
        continue

In [11]:
len(results)

6389

In [14]:
results[115]['text']

u"RT @Scipionista: Bordeaux ce soir c'est la bataille finale de la saison 6 de Game Of Thrones. Affligem Bolton VS Guinness Stark. https://t.\u2026"

The twitter api returns an array of hastags, let's organize this into a set of all the hashtags

In [39]:
results[0]['entities']['hashtags']

[{u'indices': [0, 5], u'text': u'vinb'}]

In [89]:
count = Counter()
for result in results[:5]:
    result_hashtags = result['entities']['hashtags']
    print result_hashtags

[{u'indices': [0, 5], u'text': u'vinb'}]
[]
[]
[{u'indices': [95, 121], u'text': u'AndaLevantaCanalFiestaV17'}]
[{u'indices': [127, 134], u'text': u'9Today'}]


In [71]:
from collections import Counter


In [90]:
def get_hashtags(results):
    count = Counter()
    for result in results:
        try:
            result_hashtags = result['entities']['hashtags']
            for hashtag in result_hashtags:
                count[hashtag['text'].lower()] += 1
        except:
            pass
    return count

In [91]:
hashtags_count = get_hashtags(results)
hashtags_count.most_common()

[(u'guinness', 315),
 (u'andalevantacanalfiestav17', 131),
 (u'fathersday', 113),
 (u'atoasttodad', 81),
 (u'video', 78),
 (u'dublin', 70),
 (u'virales', 69),
 (u'ireland', 49),
 (u'laurelandhardy', 47),
 (u'bubblefootball2016', 41),
 (u'beer', 36),
 (u'canneslions', 35),
 (u'tattoo', 35),
 (u'spazio', 34),
 (u'volare', 34),
 (u'euro2016', 29),
 (u'usopen', 28),
 (u'catsagainstbrexit', 28),
 (u'happyfathersday', 26),
 (u'novavideoz', 23),
 (u'toledo', 21),
 (u'photo', 19),
 (u'letsgo', 16),
 (u'mug', 16),
 (u'world', 15),
 (u'smaccdub', 15),
 (u'records', 14),
 (u'imagine', 14),
 (u'craziest', 14),
 (u'batalladecampeonas', 13),
 (u'boxeo', 13),
 (u'selfies', 12),
 (u'wnba', 11),
 (u'travel', 10),
 (u'siguemeytesigo', 10),
 (u'youvs', 10),
 (u'guiness', 10),
 (u'jam\xf3n', 10),
 (u'whataguy', 9),
 (u'legend', 9),
 (u'mydayinla', 9),
 (u'friend', 9),
 (u'dadjokes', 9),
 (u'darcc16', 9),
 (u'brexit', 8),
 (u'smaccsaveslives', 8),
 (u'25marat\xf3ncuentos', 8),
 (u'food', 8),
 (u'rt', 8),
 

In [105]:
df = pd.DataFrame.from_dict(hashtags_count,orient='index').reset_index()
df.rename(columns={'index':'hashtag', 0:'count'},inplace=True)
df.sort_values(by='count',ascending=False)

,hashtag,count
417,guinness,315
896,andalevantacanalfiestav17,131
278,fathersday,113
248,atoasttodad,81
73,video,78
123,dublin,70
226,virales,69
676,ireland,49
745,laurelandhardy,47
527,bubblefootball2016,41


In [106]:
df.head()

,hashtag,count
0,worldrecord,4
1,raining,1
2,breathnachfamily,1
3,gastronomía,3
4,alf,1


In [122]:
def get_hash_corpus(row):
    row = row.lower()
    corpus = u''
    for result in results:
        hashtag_list = []
        try:
            for sub_tag in result['entities']['hashtags']:
                hashtag_list.append(sub_tag['text'].lower())
            if row in hashtag_list:
                corpus += result['text']
        except:
            pass
    return corpus

In [123]:
df['corpus'] = [get_hash_corpus(row) for row in df['hashtag']]

In [125]:
df.head()

,hashtag,count,corpus
0,worldrecord,4,Final trials #GuinnessWorldRecords #WorldRecor...
1,raining,1,#Sunday #family #nightsin #Guinness #perfect #...
2,breathnachfamily,1,#dublin #walshfamilyvacation #breathnachfamily...
3,gastronomía,3,RT @diariodeavisos: 🍴 Toledo entra en el Guin...
4,alf,1,"#1990 im Guinness Buch, heute tot. #alf https:..."


In [126]:
pd.to_pickle(df,'guiness_hashed.pkl')

In [5]:
# df = pd.read_pickle('budlight_hashed.pkl')

In [127]:
df.tail()

,hashtag,count,corpus
1051,asseenincolumbus,2,0.56 ml of Guinness is lost in the drinker's f...
1052,entérate,1,RT @El_Universal_Mx: #Entérate @CD9 conquista ...
1053,princesssaoirse,1,#dublin #walshfamilyvacation #breathnachfamily...
1054,original,1,#Guinness\n#Original\n#Stout\n#Beer\n#Instabee...
1055,songs,1,#Guinness #Commercials #Songs https://t.co/qCV...


We now have a data frame where the 'hashtags' column has all the hastags mentioned in the budlight twitter query, and 'corpus' has all the text from all the tweets containing the respective hashtag.

This will essentially tell us which text is used in relation to each hashtag, we can now use tfidf and Latent Semantic Analysis to find common topics

For this portion I choose to use the Textacy library (which is built on spaCy) for a few reasons:
- Textacy/spaCy run in Cython, so it is high-performing.  This import because the application will eventually be user-facing
- Textacy/spaCy can handle odd terms such as emojis
- Textacy has an implementation for TFIDF (among other features), which might be import for filtering out unimport words in the terms matrix
- Textacy has Latent Semantic Analysis built it

In [128]:
import textacy

/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/numpy/lib/utils.py:99: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)


In [129]:
docs = df['corpus']

In [130]:
corpus = textacy.TextCorpus.from_texts('en',docs)

In [132]:
corpus

TextCorpus(1056 docs; 62348 tokens)

In [176]:
doc_term_matrix, id2term = corpus.as_doc_term_matrix((doc.as_terms_list(words=True, ngrams=(2,3), named_entities=True)for doc in corpus),weighting='tfidf', normalize=True, smooth_idf=True, min_df=2, max_df=0.95)

In [177]:
print(repr(doc_term_matrix))

<1056x5840 sparse matrix of type '<type 'numpy.float64'>'
	with 23699 stored elements in Compressed Sparse Row format>


Now that the tfidf matrix has been, built, we can do topic modeling to see if we can gain any insights on our groups of tweeters.  At this point we can choose between Latent Semantic Analysis (LSA), Latent Dirichlet Allocation (LDA), and Non-negative Matrix Factorization (NMF).  Here is the breakdown for why I will use LSA:

- NMF is generally suited for non text data such as pixels, and NMF can return incoherent topics
- LDA provides a topical distribution for each topic and term and often is more accurate than LSA
- LSA performs Singular Value Decomposition (SVD), which reduces the dimensionality of our already extremely large matrix.  LSA will also execute more quickly which is again important for the end user

In [178]:
model = textacy.tm.TopicModel('lsa', n_topics=10)

In [179]:
model.fit(doc_term_matrix)

In [180]:
doc_topic_matrix = model.transform(doc_term_matrix)

In [181]:
print(doc_topic_matrix.shape)

(1056, 10)


In [182]:
for topic_idx, top_terms in model.top_topic_terms(id2term, top_n=10):
    print('topic', topic_idx, ':', '   '.join(top_terms))

('topic', 0, ':', u'guinness   world   guinness world   record   world record   guinness world record   records   world records   rt   guinness world records')
('topic', 1, ':', u'dublin   guinness   ireland   storehouse   haka   irishrugby   accommodation   shouldertoshoulder   visitireland   rugbyfans')
('topic', 2, ':', u'del   toledo   m\xe1s   el   mundo   del mundo   grande   r\xe9cord   r\xe9cord guinness   plato')
('topic', 3, ':', u'guinness   drink   not   hair   beer   jasmine   duke   topless   wet   brunette')
('topic', 4, ':', u'lol   strongbow   xbox https://t.co/alaxat4amm   https://t.co/alaxat4amm   botsquad   geek   xbox   podcast   gamers   oreos')
('topic', 5, ':', u'records   world records   guinness world records   160511   got7   guinness   local   england   pub   got7district')
('topic', 6, ':', u'jasmine   duke   wet   topless   brunette   hair   drink   glamour   long   records')
('topic', 7, ':', u'england   local   village   watch the game   publife   saxon 

For the above we used 10 topics so that we can show what is happening in a human readable format, but for the next steps we will want to use more topics

In [183]:
model = textacy.tm.TopicModel('lsa', n_topics=100)
model.fit(doc_term_matrix)
doc_topic_matrix = model.transform(doc_term_matrix)
print(doc_topic_matrix.shape)

(1056, 100)


Now that we have our reduced matrix, we can do some clustering to see what hashtags are related

Because we don't want the user to have to choose the number of clusters, we have three clustering options from Sklearn that don't require a n_clusters paramter:
- DBSCAN generally performs better with more data
- MeanShift might perform well but is too computationally intensive
- Affinity Prograpagation is the most realistic option here

In [184]:
from sklearn.cluster import AffinityPropagation
ap = AffinityPropagation()
%time ap.fit(doc_topic_matrix)
clusters = ap.labels_.tolist()

CPU times: user 2.05 s, sys: 88 ms, total: 2.14 s
Wall time: 2.04 s


Now that we have the list of clusters, let's connect them with our hashtags to see what hashtags are being grouped together

In [185]:
df.head()

,hashtag,count,corpus,clusters,cluster_count,cluster
0,worldrecord,4,Final trials #GuinnessWorldRecords #WorldRecor...,74,17,1
1,raining,1,#Sunday #family #nightsin #Guinness #perfect #...,32,2154,0
2,breathnachfamily,1,#dublin #walshfamilyvacation #breathnachfamily...,0,2154,1
3,gastronomía,3,RT @diariodeavisos: 🍴 Toledo entra en el Guin...,81,10,1
4,alf,1,"#1990 im Guinness Buch, heute tot. #alf https:...",68,2154,1


In [186]:
df['cluster'] = clusters

In [187]:
df.head()

,hashtag,count,corpus,clusters,cluster_count,cluster
0,worldrecord,4,Final trials #GuinnessWorldRecords #WorldRecor...,74,17,76
1,raining,1,#Sunday #family #nightsin #Guinness #perfect #...,32,2154,32
2,breathnachfamily,1,#dublin #walshfamilyvacation #breathnachfamily...,0,2154,34
3,gastronomía,3,RT @diariodeavisos: 🍴 Toledo entra en el Guin...,81,10,72
4,alf,1,"#1990 im Guinness Buch, heute tot. #alf https:...",68,2154,76


In [188]:
df['cluster_count'] = [df['count'][df['cluster'] == row].sum() for row in df['count']]

In [189]:
df.head()

,hashtag,count,corpus,clusters,cluster_count,cluster
0,worldrecord,4,Final trials #GuinnessWorldRecords #WorldRecor...,74,27,76
1,raining,1,#Sunday #family #nightsin #Guinness #perfect #...,32,5,32
2,breathnachfamily,1,#dublin #walshfamilyvacation #breathnachfamily...,0,5,34
3,gastronomía,3,RT @diariodeavisos: 🍴 Toledo entra en el Guin...,81,4,72
4,alf,1,"#1990 im Guinness Buch, heute tot. #alf https:...",68,5,76


In [190]:
top_ten = df['cluster_count'].value_counts()[:10]

In [191]:
top_ten

5      673
6      188
4       81
27      40
15      18
173     15
8       12
16       9
11       3
12       3
Name: cluster_count, dtype: int64

In [192]:
top_ten.index

Int64Index([5, 6, 4, 27, 15, 173, 8, 16, 11, 12], dtype='int64')

In [193]:
for i in top_ten.index:
    print 'Cluster #: ' + str(i) + ' Total Tweeters: ' + str(top_ten.ix[i])
    print df['hashtag'][df.clusters == i].values[:10]

Cluster #: 5 Total Tweeters: 673
[u'letsblamecanada' u'banff' u'gooddayforaguinness' u'guinnessisgoodforyou']
Cluster #: 6 Total Tweeters: 188
[u'whataguy' u'legend' u'friend']
Cluster #: 4 Total Tweeters: 81
[u'lille' u'ninews' u'ueafa' u'irlandais']
Cluster #: 27 Total Tweeters: 40
[u'nottingham' u'totallyworthitthough' u'ringthebell' u'keeping'
 u'sl\xe1inte' u'oldmantroy' u'folkestoneharbour' u'worldsbestdad2016'
 u'hotsalvationrecords' u'finnmccools']
Cluster #: 15 Total Tweeters: 18
[u'jacksonwang' u'got7district' u'got7' u'\uac13\uc138\ube10'
 u'\uc7ad\uc2a8' u'mark' u'bambam' u'got7flyfortheworld' u'markbam']
Cluster #: 173 Total Tweeters: 15
[]
Cluster #: 8 Total Tweeters: 12
[u'facebook' u'webmaster' u'startups' u'wordpress']
Cluster #: 16 Total Tweeters: 9
[u'eventprofs' u'gravitybar' u'architecture' u'gsh' u'event']
Cluster #: 11 Total Tweeters: 3
[u'\u6311\u6226\u4e2d' u'\u30ae\u30cd\u30b9' u'\u5cb8\u548c\u7530'
 u'\u4e16\u754c\u4e00\u304b\u3057\u307f\u3093\u98df\u3079\u30

Let's try this all again with DBSCAN

In [195]:
from sklearn.cluster import DBSCAN
dbs = DBSCAN()
%time dbs.fit(doc_topic_matrix)
clusters = dbs.labels_.tolist()
df['cluster'] = clusters
df['cluster_count'] = [df['count'][df['cluster'] == row].sum() for row in df['count']]
top_ten = df['cluster_count'].value_counts()[:10]
for i in top_ten.index:
    print 'Cluster #: ' + str(i) + ' Total Tweeters: ' + str(top_ten.ix[i])
    print df['hashtag'][df.cluster == i].values[:10]

CPU times: user 461 ms, sys: 10.5 ms, total: 471 ms
Wall time: 546 ms
Cluster #: 8 Total Tweeters: 681
[u'thenook' u'stwishy' u'nofilter' u'nofuckinneed' u'stwishaloffical']
Cluster #: 11 Total Tweeters: 185
[u'specials' u'burgertheorykci' u'nkc' u'burgertheory' u'kc']
Cluster #: 17 Total Tweeters: 72
[u'\u9154\u3063\u6255\u3044'
 u'\u5bdd\u306a\u3044\u3088\u3046\u306b\u9811\u5f35\u308b' u'abbotschoice'
 u'\u6c60\u888b' u'\u958b\u6f14\u5f85\u3061']
Cluster #: 9 Total Tweeters: 40
[u'ghostbusters' u'hitlerdroge' u'gifparty' u'worldbookday' u'biathlon']
Cluster #: 5 Total Tweeters: 23
[u'dukes' u'jasmine' u'brunette' u'long' u'topless' u'drink' u'wet' u'hair'
 u'glamour']
Cluster #: 0 Total Tweeters: 19
[u'worldrecord' u'breathnachfamily' u'gastronom\xeda' u'alf' u'birdcage'
 u'fountainpub' u'womensart' u'nottingham' u'euro16gr' u'follow']
Cluster #: 37 Total Tweeters: 18
[]
Cluster #: 16 Total Tweeters: 7
[u'\uc0c8\uc6b0\uce69' u'\uc77c\uc694\uc77c' u'\ud63c\ub9e5'
 u'\uc220\uc2a4\ud0c0

In [36]:
from sklearn.metrics.pairwise import cosine_similarity
%time dist = 1 - cosine_similarity(doc_topic_matrix)
from sklearn.cluster import DBSCAN
dbs = DBSCAN(metric='precomputed')
%time dbs.fit(dist)
clusters = dbs.labels_.tolist()
df['clusters'] = clusters
top_ten = df['clusters'].value_counts()[:10]
for i in top_ten.index:
    print 'Cluster #: ' + str(i) + ' Hashtags: ' + str(top_ten.ix[i])
    print df['hashtags'][df.clusters == i].values[:10]

CPU times: user 11.3 ms, sys: 2.13 ms, total: 13.4 ms
Wall time: 12 ms
Cluster #: 0 Hashtags: 562
[u'Canada' u'Friday' u'justsayin' u'dance' u'hats' u'BuildABar'
 u'boersandbernsteinshow' u'whatsmakeup' u'Pride2016' u'Pride']
Cluster #: -1 Hashtags: 62
[u'cooliecup' u'yall' u'jackdaniels' u'jarritos' u'kmelsummerjam' u'twins'
 u'germanfest' u'finallyfuckinleft' u'ionaz'
 u'UMustStartDrinkingEarlyAzFuck']
Cluster #: 8 Hashtags: 13
[u'Ebay' u'eBay' u'Collectible' u'Discount' u'Sales' u'Basketball'
 u'Auction' u'Buzz' u'NBA' u'Detroit']
Cluster #: 16 Hashtags: 12
[u'Summer' u'ribfest2016' u'downforwhatever' u'summer' u'InThrees'
 u'LimeaRita' u'TheLodge' u'RibFest' u'surfyogabeer' u'yoga']
Cluster #: 3 Hashtags: 12
[u'jamaica' u'Pasadena' u'MexicoVsJamaica' u'Mexico' u'soccer' u'stencil'
 u'Rosebowl' u'mexico' u'AtlanticCity' u'doac']
Cluster #: 1 Hashtags: 12
[u'SundayFunday' u'WhatYeeKnow' u'sundayfunday' u'atl' u'Pizza' u'pizza'
 u'vezzo' u'ATL' u'LikeATank' u'Cobb']
Cluster #: 15 Hash

And once more with MeanShift

In [40]:
from sklearn.cluster import MeanShift
ms = MeanShift()
%time ms.fit(doc_topic_matrix)
clusters = ms.labels_.tolist()
df['clusters'] = clusters
top_ten = df['clusters'].value_counts()[:10]
for i in top_ten.index:
    print 'Cluster #: ' + str(i) + ' Hashtags: ' + str(top_ten.ix[i])
    print df['hashtags'][df.clusters == i].values[:10]

CPU times: user 2.54 s, sys: 61.4 ms, total: 2.6 s
Wall time: 2.78 s
Cluster #: 0 Hashtags: 722
[u'Canada' u'Friday' u'justsayin' u'dance' u'hats' u'SundayFunday'
 u'BuildABar' u'boersandbernsteinshow' u'cooliecup' u'whatsmakeup']
Cluster #: 3 Hashtags: 5
[u'rebelt5' u'teamcanon' u'canonrebel' u'beachmusic' u'northhills']
Cluster #: 2 Hashtags: 5
[u'GameDay' u'surfing' u'surf' u'sunshine' u'SunGlasses']
Cluster #: 1 Hashtags: 5
[u'mural' u'onmytravel' u'onmyrte66' u'Springfield' u'illinois']
Cluster #: 5 Hashtags: 4
[u'twins' u'Brazilian' u'Rio' u'Bitburger']
Cluster #: 4 Hashtags: 4
[u'fuckedup' u'cansorbottels' u'paperorplastic' u'themen']


Notes:
- 6/15 Used textacy and setup LSA, but it doesn't seem to return anything actionable, going to try clustering by cosine similarity

Connect tweets, tweeter counts, work on summarizer 

add user ids for direct marketing